In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import preprocessing
import numpy as np
from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
from gensim.models import Word2Vec
import warnings
import collections
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import random
from tqdm import tqdm


def GenerateAX(CITY):

    city=CITY

    def ODtoA(OD,INDEX):
        dic_id=pd.read_csv(f'/demo_data//dic_id_{city} city.csv')

        N_cbg=len(dic_id)

        INDEX=INDEX
        M=pd.concat([ OD[['O']],OD[['D']]],axis=0)
        M=M.fillna(0)

        m = n = N_cbg
        Mat = [[0] * m] * n
        Mat_np=np.array(Mat)
        for i in range(len(OD)):
            O=OD.iloc[i,:]['O_id']
            D=OD.iloc[i,:]['D_id']
            Mat_np[O,D]=OD.iloc[i,:]['flow']
        row, col = np.diag_indices_from(Mat_np)
        Mat_np[row,col] = 0

        m = n = N_cbg
        Mat_sym = [[0] * m] * n
        Mat_sym=np.array(Mat_sym)

        for i in range(N_cbg):
            for j in range(N_cbg):
                Mat_sym[i,j]=Mat_np[i,j]+Mat_np[j,i]

        A = csr_matrix(Mat_sym)
        print("Sparse matrix: \n",A)

        m = 1
        n=N_cbg

        sparse.save_npz('/demo_data/A_'+INDEX+'.npz', A)



    plt.rcParams['font.sans-serif']=['SimHei']
    plt.rcParams['axes.unicode_minus']=False


    def get_randomwalk(G,node, path_length):


        random_walk = [node]

        for i in range(path_length-1):

            temp = list(G.neighbors(node))
            temp = list(set(temp) - set(random_walk))
            if len(temp) == 0:
                break
            random_node = random.choice(temp)
            random_walk.append(random_node)
            node = random_node

        return random_walk



    def ODtoX_node2vec(OD,INDEX):
        OD=OD.rename(columns={'flow':'weight'})

        plt.rcParams['font.sans-serif']=['SimHei']
        plt.rcParams['axes.unicode_minus']=False

        G = nx.from_pandas_edgelist(OD, "O", "D",edge_attr=['weight'], create_using=nx.Graph())
        nx.get_edge_attributes(G, "weight")
        G_ste = StellarGraph.from_networkx(G)
        walk_length = 100  # maximum length of a random walk to use throughout this notebook
        _, weights = G_ste.edges(include_edge_weight=True)

        wt, cnt = np.unique(weights, return_counts=True)
        rw = BiasedRandomWalk(G_ste)
        weighted_walks = rw.run(
            nodes=G_ste.nodes(),  # root nodes
            length=walk_length,  # maximum length of a random walk
            n=10,  # number of random walks per root node
            p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
            q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
            weighted=True,  # for weighted random walks
            seed=42,  # random seed fixed for reproducibility
        )
        print("Number of random walks: {}".format(len(weighted_walks)))
        weighted_model = Word2Vec(
            weighted_walks, vector_size=256, window=5, min_count=0, sg=1, workers=1
        )
        X = weighted_model.wv.vectors
        X.shape
        all_nodes = list(G.nodes())

        X_pd=pd.DataFrame(X)

        nodes_pd=pd.DataFrame(all_nodes)
        nodes_pd.columns=['node']
        X_deep=pd.concat([nodes_pd,X_pd],axis=1)

        dic_id=pd.read_csv(f'/demo_data/dic_id_{city} city.csv')
        dic_id.columns=['cbg','id']
        dic_id

        dic_id.set_index('cbg',inplace=True)
        X_deep.set_index('node',inplace=True)
        result=pd.concat([dic_id,X_deep],axis=1)


        result=result.fillna(0)
        result[result.isnull().T.any()]

        Xdeep_np=np.array(result)
        Xdeep = csr_matrix(Xdeep_np)
        print("Sparse matrix: \n",Xdeep)

        sparse.save_npz('/demo_data/X_node2vec_'+INDEX+'.npz', Xdeep)

    OD=pd.read_csv(f'\damo_data\\OD_{city} city.csv')
    ODtoA(OD,city)
    ODtoX_node2vec(OD,city)

D:\Anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
D:\Anaconda\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
import os
def extract_fifth_word_from_filenames(folder_path):
    fifth_words = []  # 初始化列表以收集城市名称

    # 获取文件夹中所有以.csv结尾的文件名称
    filenames = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for filename in filenames:

        if "OD_" in filename and " city.csv" in filename:

            city_with_suffix = filename.split("OD_")[1]
            city_name = city_with_suffix.split(" city")[0]
            fifth_words.append(city_name)

    return fifth_words

folder_path = 'D:\\GNN\\OCD\\Multicity\\OD\\'
fifth_words_list = extract_fifth_word_from_filenames(folder_path)
print(fifth_words_list)

['Albuquerque', 'Atlanta', 'Austin', 'Bakersfield', 'Baltimore', 'Boston', 'Charlotte', 'Chicago', 'Colorado Springs', 'Columbus', 'Dallas', 'Denver', 'Detroit', 'El Paso', 'Fort Worth', 'Fresno', 'Houston', 'Indianapolis', 'Jacksonville', 'Kansas City', 'Las Vegas', 'Long Beach', 'Los Angeles', 'Louisville+Jefferson County metro government', 'Memphis', 'Mesa', 'Miami', 'Milwaukee', 'Minneapolis', 'Nashville-Davidson metropolitan government', 'Oakland', 'Oklahoma City', 'Omaha', 'Philadelphia', 'Phoenix', 'Portland', 'Raleigh', 'Sacramento', 'San Antonio', 'San Diego', 'San Francisco', 'San Jose', 'Seattle', 'Tampa', 'Tucson', 'Tulsa', 'Virginia Beach', 'Washington', 'Wichita']


In [ ]:
GenerateAX("New York")

Sparse matrix: 
   (0, 5)	1
  (0, 12)	2
  (0, 13)	9
  (0, 15)	7
  (0, 19)	2
  (0, 32)	1
  (0, 33)	8
  (0, 35)	2
  (0, 36)	1
  (0, 39)	1
  (0, 44)	1
  (0, 50)	1
  (0, 79)	1
  (0, 84)	2
  (0, 88)	1
  (0, 105)	3
  (0, 119)	3
  (0, 120)	5
  (0, 176)	1
  (0, 186)	2
  (0, 193)	1
  (0, 211)	1
  (0, 212)	9
  (0, 213)	2
  (0, 215)	1
  :	:
  (6592, 6566)	7
  (6592, 6567)	5
  (6592, 6568)	3
  (6592, 6570)	956
  (6592, 6571)	36
  (6592, 6572)	206
  (6592, 6573)	49
  (6592, 6574)	44
  (6592, 6575)	64
  (6592, 6576)	64
  (6592, 6577)	7
  (6592, 6578)	52
  (6592, 6579)	6
  (6592, 6580)	25
  (6592, 6581)	300
  (6592, 6582)	94
  (6592, 6583)	463
  (6592, 6584)	156
  (6592, 6585)	243
  (6592, 6586)	2
  (6592, 6587)	97
  (6592, 6588)	482
  (6592, 6589)	205
  (6592, 6590)	387
  (6592, 6591)	172
Number of random walks: 65860
Sparse matrix: 
   (0, 1)	-0.002570265904068947
  (0, 2)	-0.0034175743348896503
  (0, 3)	0.0020152339711785316
  (0, 4)	0.003708168864250183
  (0, 5)	-0.0006605638191103935
  (0, 6)	0.

In [ ]:
for CITY in fifth_words_list:
    GenerateAX(CITY)

Sparse matrix: 
   (2, 3)	103
  (2, 4)	2531
  (2, 5)	2932
  (2, 6)	216
  (2, 7)	26
  (2, 8)	193
  (2, 9)	11
  (2, 10)	45
  (2, 11)	20
  (2, 12)	24
  (2, 13)	6
  (2, 14)	12
  (2, 15)	6
  (2, 16)	44
  (2, 17)	12
  (2, 18)	749
  (2, 19)	283
  (2, 20)	25
  (2, 21)	264
  (2, 22)	118
  (2, 23)	533
  (2, 24)	51
  (2, 25)	14
  (2, 26)	9
  (2, 27)	26
  :	:
  (712, 685)	7
  (712, 686)	5
  (712, 687)	26
  (712, 688)	2
  (712, 689)	12
  (712, 690)	14
  (712, 691)	21
  (712, 692)	35
  (712, 693)	5
  (712, 696)	17
  (712, 697)	27
  (712, 698)	11
  (712, 699)	3
  (712, 700)	16
  (712, 701)	48
  (712, 702)	11
  (712, 703)	8
  (712, 704)	23
  (712, 705)	139
  (712, 706)	14
  (712, 707)	35
  (712, 708)	29
  (712, 709)	13
  (712, 710)	10245
  (712, 711)	3393
Number of random walks: 7090
Sparse matrix: 
   (1, 0)	1.0
  (2, 0)	2.0
  (2, 1)	0.002127908170223236
  (2, 2)	-0.003182344138622284
  (2, 3)	0.0022381432354450226
  (2, 4)	-0.0013642790727317333
  (2, 5)	1.0407529771327972e-05
  (2, 6)	-0.0028706556